# 9.6.1 Asset Replacement

設定は7.6.2 Asset Replacementを引き継いでいる。

毎年初めにkeepかreplaceか選択する。a年経過後のoutputがq(a)、replacement costがc

但し、pが$p_{t+1} = h(p_{t}, ε_{t+1})$で決定される

state variables: p, a

action variables: x

Reward functionは、
    $f(p, a, x) = pq(a)$    ($x = keep$)、$f(p, a, x) = pq(0)-c$      ($x = replace$)

Value functionは、
    $V(p, a) = \max \{pq(a) + δE_ε V(h(p, ε), a+1), pq(0) - c + E_ε V(h(p, ε), 1) \}$

In [1]:
using QuantEcon
using Plots

In [2]:
function q(a)
    return 50 - 2.5 * a - 2.5 * a^2
end

q (generic function with 1 method)

In [3]:
mutable struct AssetReplacement
    price::Float64
    kbar::Float64
    gamma::Float64
    abar::Float64
    sigma::Float64
    delta::Float64
    k_vec::Vector{Float64}
    
    function AssetReplacement(price=2,
                              kbar=100,
                              gamma=0.5,
                              abar=6,
                              sigma=15,
                              delta=0.9,
                              k_vec=collect(linspace(30, 190, 100)))
        return new(price, kbar, gamma, abar, sigma, delta, k_vec)
    end
end

In [4]:
function update_bellman!(ar::AssetReplacement, V, V_new, W, W_new, X, X_new, Y, Y_new, Z, Z_new)
    price, kbar, gamma, abar, sigma, delta = ar.price, ar.kbar, ar.gamma, ar.abar, ar.sigma, ar.delta
    
    V_func = LinInterp(ar.k_vec, V) # a=1
    W_func = LinInterp(ar.k_vec, W) # a=2
    X_func = LinInterp(ar.k_vec, X) # a=3
    Y_func = LinInterp(ar.k_vec, Y) # a=4
    Z_func = LinInterp(ar.k_vec, Z) # a=5
    e, w = qnwnorm(5, 0, sigma)
    
    for (k_idx, k) in enumerate(ar.k_vec)
            V_new[k_idx] = max(price * q(1) + delta * dot(W_func.(e .+ kbar .+ (gamma * (k-kbar))), w),
                               price * q(0) - k + delta * dot(V_func.(e .+kbar .+ (gamma * (k-kbar))), w))
    end
    for (k_idx, k) in enumerate(ar.k_vec)
            W_new[k_idx] = max(price * q(2) + delta * dot(X_func.(e .+kbar .+ (gamma * (k-kbar))), w),
                               price * q(0) - k + delta * dot(V_func.(e .+kbar .+ (gamma * (k-kbar))), w))
    end
    for (k_idx, k) in enumerate(ar.k_vec)
            X_new[k_idx] = max(price * q(3) + delta * dot(Y_func.(e .+kbar .+ (gamma * (k-kbar))), w),
                               price * q(0) - k + delta * dot(V_func.(e .+kbar .+ (gamma * (k-kbar))), w))
    end
    for (k_idx, k) in enumerate(ar.k_vec)
            Y_new[k_idx] = max(price * q(4) + delta * dot(Z_func.(e .+kbar .+ (gamma * (k-kbar))), w),
                               price * q(0) - k + delta * dot(V_func.(e .+kbar .+ (gamma * (k-kbar))), w))
    end
    for (k_idx, k) in enumerate(ar.k_vec)
            Z_new[k_idx] = price * q(0) - k + delta * dot(V_func.(e .+kbar .+ (gamma * (k-kbar))), w)
    end
    
    return V_new, W_new, X_new, Y_new, Z_new
end

update_bellman! (generic function with 1 method)

In [5]:
k_vec = collect(linspace(30, 190, 100))

100-element Array{Float64,1}:
  30.0   
  31.6162
  33.2323
  34.8485
  36.4646
  38.0808
  39.697 
  41.3131
  42.9293
  44.5455
  46.1616
  47.7778
  49.3939
   ⋮     
 172.222 
 173.838 
 175.455 
 177.071 
 178.687 
 180.303 
 181.919 
 183.535 
 185.152 
 186.768 
 188.384 
 190.0   

In [6]:
V = ones(length(k_vec))
W = zeros(length(k_vec))
X = zeros(length(k_vec))
Y = zeros(length(k_vec))
Z = zeros(length(k_vec))

100-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [7]:
V_computed = copy(V)
W_computed = copy(W)
X_computed = copy(X)
Y_computed = copy(Y)
Z_computed = copy(Z)
ar = AssetReplacement()

for i in 1:500
    V, W, X, Y, Z = update_bellman!(ar, V, V_computed, W, W_computed, X, X_computed, Y, Y_computed, Z, Z_computed)
end

In [12]:
plotlyjs()

Plots.PlotlyJSBackend()

In [13]:
plot(k_vec, [V W X Y Z])